In [1]:
import pandas as pd
import torch
import transformers
import json

In [2]:
df_train = pd.read_csv("/kaggle/input/labeled-data/train_df_2100_manual_final.csv")
df_val = pd.read_csv("/kaggle/input/labeled-data/validation_df_450_manual_final.csv")
df_test = pd.read_csv("/kaggle/input/labeled-data/test_df_450_manual_final.csv")

In [3]:
checkpoint = "/kaggle/input/llama-3/transformers/8b-chat-hf/1"

pipeline = transformers.pipeline(
    "text-generation",
    model=checkpoint,
    torch_dtype=torch.float16,
    device_map="auto",
)

2024-06-27 20:55:26.106619: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-27 20:55:26.106720: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-27 20:55:26.283132: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

In [5]:
json_schema = {"classification": "classification value between 1, 0, -1",
              "explanation": "reasoning of the answer in 50 words"}

input_tweet = "Pongase serio Nayib Bukele  no nos venga con esa ud bien save que nadie a salido al espacio"

system_message = """You are an AI assistant designed to answer simple questions.
Please restrict your answer to the exact question asked and respond with a json object only.
"""

user_message = f"""You are an AI language model trained to analyze tweets and comments from the people of the country of El Salvador.
    Your job is to categorize each tweet into one of three categories based on its sentiment: 1 (positive), 0 (neutral), or -1 (negative) using a
    local perspective from El salvador political scene.

    Guidelines:
    . 1 (Positive): The tweet expresses a positive sentiment, such as happiness, praise, appreciation or admiration. Be careful of the use of sarcasm and avoid classifying sarcastic tweets as positive
    . 0 (Neutral): The tweet is neutral,. It may present facts, ask questions, or be generally balanced in tone or a statement or just mention a fact or a name.
    .-1 (Negative): The tweet expresses a negative sentiment, such as criticism, disappointment, or disapproval. Classify sarcastic comments which actually have a negative sentiment here

    Examples for Reference:

    1 (POS):
    . Tweet: "Dios lo bendiga por ser un gran ser humano."
    - Reasoning: The tweet is praising and expressing positive feelings towards someone.

    . Tweet: "Me encanta la humanidad de nuestro astronauta, un hombre con gran corazón."
    - Reasoning: The tweet shows love and admiration for the astronaut.

    . Tweet: "Dos grandes hombres haciendo historia. Gracias por todo."
    - Reasoning: The tweet appreciates the actions of two men, showing gratitude and positivity.

    0 (NEU):
    . Tweet: "Tweet Maria Alicia Alas Moreno "
    - Reasoning:The tweet states a fact about the need for a certain type of president without strong positive or negative sentiment.

    . Tweet: "Tweet Nuestros obstaculos son mentales"
    - Reasoning: The tweet presents a factual statement about the astronaut's activities without expressing an opinion.

    . Tweet: "¿Qué opinan sobre las últimas noticias del presidente?"
    - Reasoning: The tweet is asking a question and does not convey a positive or negative sentiment.

    -1 (NEG):
    . Tweet: "No estoy de acuerdo con las políticas actuales."
    - Reasoning: The tweet expresses disagreement with current policies, indicating a negative sentiment.

    . Tweet: "Es una vergüenza que esto esté sucediendo en nuestro país."
    - Reasoning: The tweet expresses disappointment and criticism about a situation in the country.

    . Tweet: "Las decisiones del presidente están dañando la economía."
    - Reasoning: The tweet criticizes the president's decisions, showing a negative sentiment about their impact on the economy.
    Instructions:
        
        - Read the tweet carefully.

        - Determine the sentiment expressed based on your expertise, the guidelines and examples provided above:

        - Generate a json output following the json schema:
    
    Tweet: {input_tweet}
    Json Schema: {json_schema}
    Your Answer: """

messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_message},
]

prompt = pipeline.tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize = False)

terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

max_length=256
temperature = 0.1

response = pipeline(prompt,
#                    do_samples=True,
#                    top_p=0.9,
        temperature=temperature,
#         num_return_sequences=1,
        eos_token_id=terminators,
        max_new_tokens=max_length,
        return_full_text=False,
                    repetition_penalty=1.0
#         pad_token_id=pipeline.model.config.eos_token_id
                   )

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [6]:
response

[{'generated_text': '{\n"classification": -1,\n"explanation": "The tweet expresses a negative sentiment towards Nayib Bukele, criticizing him for not being serious and making false claims about space travel, indicating a negative sentiment towards his leadership."\n}'}]

In [48]:
json.loads(response[0]['generated_text'])

{'classification': -1,
 'explanation': 'The tweet expresses a negative sentiment towards Nayib Bukele, criticizing him for not being serious and making false claims about space travel, indicating a negative sentiment towards his leadership.'}

## Deployment Code

In [45]:
import pandas as pd
import torch
import transformers
import json
import time

class TweetClassifier:
    def __init__(self, model_checkpoint):
        self.pipeline = transformers.pipeline(
            "text-generation",
            model=model_checkpoint,
            torch_dtype=torch.float16,
            device_map="auto",
        )
        self.system_message = """You are an AI assistant designed to answer simple questions.
Please restrict your answer to the exact question asked and respond with a json object only.
"""
        self.json_schema = {
            "classification": "classification value between 1, 0, -1",
            "explanation": "reasoning of the answer in 50 words"
        }

    def create_prompt(self, tweet):
        user_message = f"""You are an AI language model trained to analyze tweets and comments from the people of the country of El Salvador.
    Your job is to categorize each tweet into one of three categories based on its sentiment: 1 (positive), 0 (neutral), or -1 (negative) using a
    local perspective from El salvador political scene.

    Guidelines:
    . 1 (Positive): The tweet expresses a positive sentiment, such as happiness, praise, appreciation or admiration. Be careful of the use of sarcasm and avoid classifying sarcastic tweets as positive
    . 0 (Neutral): The tweet is neutral,. It may present facts, ask questions, or be generally balanced in tone or a statement or just mention a fact or a name.
    .-1 (Negative): The tweet expresses a negative sentiment, such as criticism, disappointment, or disapproval. Classify sarcastic comments which actually have a negative sentiment here

    Examples for Reference:

    1 (POS):
    . Tweet: "Dios lo bendiga por ser un gran ser humano."
    - Reasoning: The tweet is praising and expressing positive feelings towards someone.

    . Tweet: "Me encanta la humanidad de nuestro astronauta, un hombre con gran corazón."
    - Reasoning: The tweet shows love and admiration for the astronaut.

    . Tweet: "Dos grandes hombres haciendo historia. Gracias por todo."
    - Reasoning: The tweet appreciates the actions of two men, showing gratitude and positivity.

    0 (NEU):
    . Tweet: "Tweet Maria Alicia Alas Moreno "
    - Reasoning:The tweet states a fact about the need for a certain type of president without strong positive or negative sentiment.

    . Tweet: "Tweet Nuestros obstaculos son mentales"
    - Reasoning: The tweet presents a factual statement about the astronaut's activities without expressing an opinion.

    . Tweet: "¿Qué opinan sobre las últimas noticias del presidente?"
    - Reasoning: The tweet is asking a question and does not convey a positive or negative sentiment.

    -1 (NEG):
    . Tweet: "No estoy de acuerdo con las políticas actuales."
    - Reasoning: The tweet expresses disagreement with current policies, indicating a negative sentiment.

    . Tweet: "Es una vergüenza que esto esté sucediendo en nuestro país."
    - Reasoning: The tweet expresses disappointment and criticism about a situation in the country.

    . Tweet: "Las decisiones del presidente están dañando la economía."
    - Reasoning: The tweet criticizes the president's decisions, showing a negative sentiment about their impact on the economy.
    Instructions:
        
        - Read the tweet carefully.

        - Determine the sentiment expressed based on your expertise, the guidelines and examples provided above:

        - Generate a json output following the json schema:
    
    Tweet: {tweet}
    Json Schema: {json_schema}
    Your Answer: """

        messages = [
            {"role": "system", "content": self.system_message},
            {"role": "user", "content": user_message},
        ]
        
        prompt = self.pipeline.tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)
        
        return prompt

    def classify_tweet(self, tweet):
        prompt = self.create_prompt(tweet)
        terminators = [
            self.pipeline.tokenizer.eos_token_id,
            self.pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
        ]
        max_length = 256
        temperature = 0.1
        
        
        attempts = 0
        max_attempts = 1
        
        while max_attempts > attempts:
            try:
                
                response = self.pipeline(
                    prompt,
#                     temperature=temperature,
                    eos_token_id=terminators,
                    max_new_tokens=max_length,
                    return_full_text=False,
                    repetition_penalty=1.0
                )
                result = json.loads(response[0]['generated_text'])
                
                return result['classification'], result['explanation']
            except (json.JSONDecodeError, Exception) as error:
                print(f"Error encountered (attempt {attempts + 1}):", error)
                attempts += 1
                time.sleep(1)  # Sleep before retrying
                    
                return 0, "Error in json parsing"
        

    def process_dataframe(self, df):
        classifications = []
        explanations = []
        for index, row in df.iterrows():
            tweet = row['tweet']
            classification, explanation = self.classify_tweet(tweet)
            classifications.append(classification)
            explanations.append(explanation)
        df['classification'] = pd.Series(classifications)
        df['explanation'] = pd.Series(explanations)

        return df



In [47]:
# Usage
if __name__ == "__main__":
    df_test = pd.read_csv("/kaggle/input/labeled-data/test_df_450_manual_final.csv")
    df_test.rename(columns = {'comment_processed_text': 'tweet'}, inplace=True)
    model_checkpoint = "/kaggle/input/llama-3/transformers/8b-chat-hf/1"

    classifier = TweetClassifier(model_checkpoint)
    df_result = classifier.process_dataframe(df_test.sample(10))
    
    df_result.to_csv("classified_tweets.csv", index=False)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Error encountered (attempt 1): CUDA out of memory. Tried to allocate 1002.00 MiB. GPU 0 has a total capacty of 14.75 GiB of which 991.06 MiB is free. Process 45429 has 13.78 GiB memory in use. Of the allocated memory 12.96 GiB is allocated by PyTorch, and 708.47 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Error encountered (attempt 1): CUDA out of memory. Tried to allocate 1002.00 MiB. GPU 0 has a total capacty of 14.75 GiB of which 991.06 MiB is free. Process 45429 has 13.78 GiB memory in use. Of the allocated memory 12.96 GiB is allocated by PyTorch, and 705.26 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF


...

Error encountered (attempt 1): CUDA out of memory. Tried to allocate 1002.00 MiB. GPU 0 has a total capacty of 14.75 GiB of which 991.06 MiB is free. Process 45429 has 13.78 GiB memory in use. Of the allocated memory 12.96 GiB is allocated by PyTorch, and 708.47 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Error encountered (attempt 1): CUDA out of memory. Tried to allocate 1002.00 MiB. GPU 0 has a total capacty of 14.75 GiB of which 991.06 MiB is free. Process 45429 has 13.78 GiB memory in use. Of the allocated memory 12.98 GiB is allocated by PyTorch, and 693.55 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Error encountered (attempt 1): CUDA out of memory. Tried to allocate 1002.00 MiB. GPU 0 has a total capacty of 14.75 GiB of which 991.06 MiB is free. Process 45429 has 13.78 GiB memory in use. Of the allocated memory 12.99 GiB is allocated by PyTorch, and 679.46 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF


In [41]:
df_result

,Unnamed: 0.1,Unnamed: 0,comment_url,comment_createdAt,comment_id,comment_viewCount,comment_lang,comment_author_createdAt,comment_location,comment_text,...,post_verified,post_text,isActor_self__decalred__location__El_Salvador,post_processed_text,actor_affiliation,llama3_output,llama3_reason,manual labeling,classification,explanation
136,728,2248,https://x.com/cesarcortezfoto/status/175132929...,1.706384e+09,1.751330e+18,16.0,es,16/01/2018,Canada,@PulsoCiudadanos @Cerg89 @ARENAOFICIAL En qué ...,...,True,"#PulsoCiudadano 🗣️| “Yo nací en democracia, cr...",True,yo nací en democracia crecí en democracia y ju...,Media,0.0,The tweet is asking a question about the count...,0,0,The tweet is asking a question about someone's...
0,2621,8340,https://x.com/az75de/status/1299484106313682947,1.598656e+09,1.299480e+18,NaN,es,30/03/2020,NaN,@FranAlabi @grupo_samu 👏🏼👏🏼👏🏼👏🏼👏🏼👏🏼👏🏼👏🏼Dios lo...,...,True,"""Nos abrazaste, no pudiste, pero nos abrazaste...",True,nos abrazaste no pudiste pero nos abrazaste he...,Government,0.0,row with error,1,1,"The tweet expresses a positive sentiment, as i..."
179,674,1946,https://x.com/MGo24332752/status/1759375768755...,1.708303e+09,1.759380e+18,4.0,es,30/08/2023,NaN,@MarceloLarin1 Larin deja de fumar 🚬 delirios...,...,True,Me asombra que aunque el gobierno hizo gran fr...,False,me asombra que aunque el gobierno hizo gran fr...,Media,-1.0,"The tweet expresses a negative sentiment, crit...",-1,-1,"The tweet expresses a negative sentiment, crit..."
433,1201,3892,https://x.com/MariaLuisaHayem/status/131791778...,1.603051e+09,1.317920e+18,NaN,es,05/06/2019,El Salvador,Nuestro Presidente @NayibBukele ha hecho una a...,...,False,"El #DespegueEconómico del país ya comenzó, las...",True,el del país ya comenzó las calificadoras de ri...,Government,1.0,The tweet expresses a positive sentiment as it...,1,1,"The tweet expresses a positive sentiment, prai..."
119,1061,3459,https://x.com/DobleH1994/status/14771509984184...,1.641015e+09,1.477150e+18,NaN,es,30/09/2011,Sivar,@easegura He aquí la definición de gata angora 🤦,...,False,"Honestamente, creo que no urgía hacer una bibl...",True,honestamente creo que no urgía hacer una bibli...,"['Media', 'Academia']",0.0,The tweet presents a factual definition of 'ga...,0,0,The tweet presents a factual definition of a c...
402,2776,8710,https://x.com/Heriberto__Es/status/16607477273...,1.684788e+09,1.660750e+18,588.0,es,16/08/2017,Ex Republica de El Salvador,"@FranAlabi Asi como despidieron a las dras, es...",...,True,Pido disculpas a la población por mis declarac...,True,pido disculpas a la población por mis declarac...,Government,-1.0,The tweet expresses a negative sentiment towar...,-1,-1,"The tweet expresses a negative sentiment, crit..."
392,815,2608,https://x.com/Lopez99715323/status/12309280493...,1.582311e+09,1.230930e+18,NaN,es,03/09/2018,"El Salvador, Zona Paracentral.",@ChristianR117 @chelita_NI @nayibbukele Es una...,...,False,⚠️ULTIMA HORA⚠️\n\nPresidente de El Salvador @...,True,ultima hora presidente de el salvador somete a...,Media,1.0,The tweet expresses a positive sentiment by me...,-1,0,The tweet presents a factual statement about t...
409,2377,7702,https://x.com/GuevaDave2108/status/17635813739...,1.709306e+09,1.763580e+18,3.0,es,05/08/2023,NaN,@MarceloLarin1 Que te importa seroton,...,True,Estimado Diputado Walter Aleman usted que opin...,False,estimado diputado walter aleman usted que opin...,Media,0.0,"The tweet is neutral, as it is a question and ...",-1,0,"The tweet is neutral, as it is a question and ..."
411,2193,6968,https://x.com/DelmerZavala2/status/13916304377...,1.620626e+09,1.391630e+18,NaN,es,01/10/2017,Honduras,@FranAlabi @nayibbukele El pueblo Hondureño es...,...,True,La solidaridad es un valor que nos debe caract...,True,la solidaridad es un valor que nos debe caract...,Government,1.0,The tweet expresses gratitude and appreciation...,1,1,The tweet expresses gratitude and appreciation...
445,2443,7979,https://x.com/DerekSm18831275/status/178149528.